In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.optimizers import Adam
from keras.layers import GRU, Dense, Input, TimeDistributed, Flatten, Dropout
from keras.models import Sequential, Model
from keras.layers import Conv2D, BatchNormalization, LeakyReLU, Reshape, Conv2DTranspose
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from keras.utils.vis_utils import plot_model

In [11]:
def make_discriminator_model(input_shape):
    model = tf.keras.Sequential()
    
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', input_shape=input_shape))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))               
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))
    
    model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    
    return model  
input_shape=64,64,3
test_discriminator_model= make_discriminator_model(input_shape)
print(test_discriminator_model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 64, 64, 128)       3584      
                                                                 
 batch_normalization_20 (Bat  (None, 64, 64, 128)      512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 64, 64, 128)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 128)       262272    
                                                                 
 batch_normalization_21 (Bat  (None, 32, 32, 128)      512       
 chNormalization)                                                
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 32, 32, 128)      

In [12]:
def make_generator_model():
    codings_size = 100
    model = Sequential()
    model.add(Dense(16*16*128, use_bias=False, input_shape=[codings_size]))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Reshape((16, 16, 128)))
    assert model.output_shape == (None, 16, 16, 128)

    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same'))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv2DTranspose(128, (5, 5), strides=(1,1), padding='same'))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv2DTranspose(128, (5, 5), strides=(2,2), padding='same'))
    assert model.output_shape == (None, 64, 64, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv2D(3, (5, 5), strides=(1, 1), padding='same', activation="tanh"))
    assert model.output_shape == (None, 64, 64, 3)

    return model
test_generator_model= make_generator_model()
print(test_generator_model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 32768)             3276800   
                                                                 
 batch_normalization_25 (Bat  (None, 32768)            131072    
 chNormalization)                                                
                                                                 
 leaky_re_lu_25 (LeakyReLU)  (None, 32768)             0         
                                                                 
 reshape_2 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 32, 32, 128)      262272    
 ranspose)                                                       
                                                                 
 batch_normalization_26 (Bat  (None, 32, 32, 128)     

In [13]:
# Function to create integrated GRU and DCGAN Model 
def GRU_DCGAN_Model(input_shape_discriminator, input_shape_gru):
    discriminator_input = Input(shape=input_shape_discriminator)
    discriminator_output = make_discriminator_model(input_shape_discriminator)(discriminator_input)
    gru_input = Input(shape=input_shape_gru)
    gru_output = GRU(64, activation="tanh", recurrent_activation="sigmoid")(gru_input)
    gru_outputd = Dropout(rate=0.2)(gru_output)
    # Concatenate the output of CNN and GRU
    merged = tf.keras.layers.concatenate([discriminator_output, gru_outputd])
    x = Dense(64, activation='relu')(merged)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[discriminator_input, gru_input], outputs=output)
    return Model(inputs=[discriminator_input, gru_input], outputs=output)

# Get the summary of the model
fixed_seq_length=10
input_shape_discriminator = (64, 64, 3)
input_shape_gru = (fixed_seq_length, 1)
model = GRU_DCGAN_Model(input_shape_discriminator, input_shape_gru)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10, 1)]      0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 gru (GRU)                      (None, 64)           12864       ['input_2[0][0]']                
                                                                                                  
 sequential_8 (Sequential)      (None, 1)            1057281     ['input_1[0][0]']                
                                                                                            

In [15]:
# Create generator model
generator=make_generator_model()
generator_optimizer = Adam(learning_rate=0.0001, beta_1=0.5)
generator.compile(loss='mean_squared_error',optimizer = generator_optimizer)
# Create discriminator model
gru_DCGAN_Model = GRU_DCGAN_Model(input_shape_discriminator, input_shape_gru)
discriminator_optimizer = Adam(learning_rate=0.0001, beta_1=0.5)
# Compile discriminator model
gru_DCGAN_Model.compile(optimizer=discriminator_optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Create gan_model model
gan_model = make_gan_model(generator, gru_DCGAN_Model, input_shape_discriminator, input_shape_gru)
gan_optimizer = Adam(learning_rate=0.0001, beta_1=0.5)
# Compile gan_model model
gan_model.compile(optimizer=gan_optimizer, loss="binary_crossentropy", metrics=["accuracy"])

TypeError: GRU_DCGAN_Model() takes 2 positional arguments but 4 were given